In [ ]:
import pandas as pd
import random
import numpy as np
from random import randint
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BertConfig, BertModel
import mteb
from sentence_transformers import SentenceTransformer, models


import os
import pickle
import time
import memory_profiler
from tqdm.notebook import tqdm

%load_ext memory_profiler

from pathlib import Path
import distro

%load_ext watermark

2025-03-14 09:54:30.552262: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 09:54:30.563371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741942470.574589  455916 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741942470.578058  455916 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 09:54:30.592443: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# old one '1.8.1+cu111'
torch.__version__

'2.5.0+cu124'

In [ ]:
%load_ext autoreload
%autoreload 2

from text_embeddings_src.legacy.model_stuff import (
    fix_all_seeds,
    train_loop,
    train_loop_without_eval,
    train_loop_without_eval_embedding_layer,
    EmbeddingOnlyModel,
    MyEmbeddingSentenceModel,
)
from text_embeddings_src.data_stuff import (
    MultOverlappingSentencesPairDataset,
)
from text_embeddings_src.legacy.embeddings import generate_embeddings

In [ ]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [ ]:
variables_path = Path("../../results/variables")
figures_path = Path("../../results/figures/updated_dataset")
data_path = Path("../../data")

In [ ]:
# MANUAL FIX TO PATH ISSUE FROM VSCODE
import text_embeddings_src

nb_path = Path(text_embeddings_src.__path__[0]).parents[0] / Path(
    "scripts/updated_dataset"
)
assert nb_path.exists(), "The path does not exist"

variables_path = (nb_path / variables_path).resolve(strict=True)
figures_path = (nb_path / figures_path).resolve(strict=True)
data_path = (nb_path / data_path).resolve(strict=True)

In [ ]:
%watermark -a 'Rita González-Márquez' -t -d -tz -u -v -iv -w -m -h -p transformers -p openTSNE
print(distro.name(pretty=True))

Author: Rita González-Márquez

Last updated: 2025-03-14 09:55:30CET

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.31.0

openTSNE: 1.0.2

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 4.18.0-553.el8_10.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 64
Architecture: 64bit

Hostname: rgonzalesmarquez_GPU0-llm_gber7

transformers         : 4.45.2
pandas               : 2.2.3
tqdm                 : 4.66.4
memory_profiler      : 0.61.0
jupyter_black        : 0.4.0
distro               : 1.9.0
numpy                : 1.26.4
sentence_transformers: 3.3.0
text_embeddings_src  : 0.0.0
mteb                 : 1.19.9
torch                : 2.5.0
black                : 24.10.0

Watermark: 2.5.0

Ubuntu 24.04 LTS


# Import ICLR dataset

In [ ]:
%%time
iclr2024 = pd.read_parquet(
    data_path / "iclr2024.parquet.gzip",
    engine="pyarrow",
)

CPU times: user 277 ms, sys: 83.3 ms, total: 360 ms
Wall time: 362 ms


In [ ]:
iclr2024.keywords = iclr2024.keywords.transform(lambda x: list(x))
iclr2024.scores = iclr2024.scores.transform(lambda x: list(x))

In [ ]:
iclr2024

,index,year,id,title,abstract,authors,decision,scores,keywords,gender-first,gender-last,t-SNE x,t-SNE y
0,0,2017,S1VaB4cex,FractalNet: Ultra-Deep Neural Networks without...,We introduce a design strategy for neural netw...,"Gustav Larsson, Michael Maire, Gregory Shakhna...",Accept (Poster),"[5, 7, 6, 6]",[],male,male,-28.117955,-20.418127
1,1,2017,H1W1UN9gg,Deep Information Propagation,We study the behavior of untrained neural netw...,"Samuel S. Schoenholz, Justin Gilmer, Surya Gan...",Accept (Poster),"[8, 9, 8]","[theory, deep learning]",male,None,-32.466820,-10.791123
2,2,2017,r1GKzP5xx,Recurrent Normalization Propagation,We propose a LSTM parametrization that preser...,"César Laurent, Nicolas Ballas, Pascal Vincent",Invite to Workshop Track,"[4, 6, 6]","[deep learning, optimization]",None,male,3.504240,19.946053
3,3,2017,S1J0E-71l,Surprisal-Driven Feedback in Recurrent Networks,Recurrent neural nets are widely used for pred...,"K, a, m, i, l, , R, o, c, k, i",Reject,"[3, 4, 3]","[unsupervised learning, applications, deep lea...",None,None,4.553473,16.037763
4,4,2017,SJGCiw5gl,Pruning Convolutional Neural Networks for Reso...,We propose a new formulation for pruning convo...,"Pavlo Molchanov, Stephen Tyree, Tero Karras, T...",Accept (Poster),"[6, 7, 9]","[deep learning, transfer learning]",None,male,-25.827705,-37.891772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24342,7299,2024,1bbPQShCT2,I-PHYRE: Interactive Physical Reasoning,Current evaluation protocols predominantly ass...,,,[],"[intuitive physics, physical reasoning]",None,None,43.137120,44.316133
24343,7300,2024,Ny150AblPu,EXPOSING TEXT-IMAGE INCONSISTENCY USING DIFFUS...,In the battle against widespread online misinf...,,,[],"[mis-contextualization, media forensic]",None,None,59.742172,-22.673627
24344,7301,2024,ZGBOfAQrMl,Video Super-Resolution Transformer with Masked...,"Recently, Vision Transformer has achieved grea...",,,[],"[video super-resolution, adaptive, memory and ...",None,None,57.933273,-3.932825
24345,7302,2024,J2kRjUAOLh,Contrastive Predict-and-Search for Mixed Integ...,Mixed integer linear programs (MILP) are flex...,,,[],[mixed integer programs; contrastive learning],None,None,-11.437999,21.289523


In [ ]:
labels_iclr = np.load(variables_path / "updated_dataset" / "labels_iclr.npy")
colors_iclr = np.load(variables_path / "updated_dataset" / "colors_iclr.npy")

pickle_in = open(
    variables_path / "updated_dataset" / "dict_label_to_color.pkl", "rb"
)
dict_label_to_color = pickle.load(pickle_in)

In [ ]:
labels_iclr

array(['unlabeled', 'unlabeled', 'optimization', ..., 'unlabeled',
       'unlabeled', 'federated learning'], dtype='<U34')

# Benchmarks

## Tasks

In [ ]:
tasks = mteb.get_tasks(
    tasks=[
        "ArxivClusteringP2P",
        "BiorxivClusteringP2P",
        "MedrxivClusteringP2P",
        "RedditClusteringP2P",
        # "TwentyNewsgroupsClustering",
        "StackExchangeClusteringP2P",
        "SciDocsRR",
        "MindSmallReranking",
        "SCIDOCS",
        "ArguAna",
        "STS12",
        "STS13",
        "STS14",
        "STS15",
        "STS16",
        "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

### Pre-trained MPNet

In [ ]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [ ]:
# pretrained_mpnet = AutoModel.from_pretrained(model_paths[0])
pretrained_mpnet = SentenceTransformer(model_paths[0])

Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
type(pretrained_mpnet)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [ ]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_mpnet,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Clustering

- StackExchangeClusteringP2P, p2p

Clustering: 100%|██████████████████████████████████████████████████████████████| 10/10 [06:37<00:00, 39.76s/it]


In [ ]:
print(results[0].scores["test"][0]["main_score"])

0.26347092269579386


### Fine-tuned MPNet (crops)

#### Fine-tune model and save
We repeat the standard finetuning to be able to save the checkpoints

In [ ]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [ ]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
print("Model: ", model_names[0])

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained(model_paths[0])
model = AutoModel.from_pretrained(model_paths[0])


# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract,
    tokenizer,
    device,
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
model_trained_crops, losses = train_loop_without_eval(
    model,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
)

# Save the model
output_dir = Path("updated_dataset/model_checkpoints/mpnet_crops_finetuning")
model_trained_crops.save_pretrained(variables_path / output_dir)


print("----------------------------")

Model:  MPNet
Running on device: cuda


/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


microsoft/mpnet-base


  0%|          | 0/368 [00:00<?, ?it/s]

----------------------------
CPU times: user 6min 6s, sys: 4min 5s, total: 10min 12s
Wall time: 5min 15s


#### Evaluate

In [ ]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [ ]:
# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet = SentenceTransformer(modules=new_modules)

In [ ]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_crops_finetuning",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Clustering

- StackExchangeClusteringP2P, p2p

Clustering:  90%|████████████████████████████████████████████████████████▋      | 9/10 [05:17<00:27, 27.56s/it]

In [ ]:
print(results[0].scores["test"][0]["main_score"])

0.3831448182125486


#### More tasks

In [ ]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [ ]:
# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet = SentenceTransformer(modules=new_modules)

In [ ]:
new_tasks = [
    "AskUbuntuDupQuestions",  # reranking
    "NFCorpus",  # retrieval, ordered by relevance
    "NQ",
    "TRECCOVID",
    "SciFact",
]

In [ ]:
tasks = mteb.get_tasks(tasks=new_tasks)
evaluation = mteb.MTEB(tasks=tasks)

In [ ]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_crops_finetuning",
    
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Reranking

- AskUbuntuDupQuestions, s2s

Retrieval

- NFCorpus, s2p

...

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

In [ ]:
for task in results:
    print(task, task.scores["test"][0]["main_score"])

In [ ]:
import os
import json


def load_mteb_results(directory):
    results = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    data = json.load(f)

                # Create a key based on the relative path
                relative_path = os.path.relpath(file_path, directory)
                key = os.path.splitext(relative_path)[
                    0
                ]  # Remove the .json extension
                results[key] = data

    return results

In [ ]:
def create_mteb_dict(original_json_dict):
    clean_dict = dict()
    for key in np.sort(list(original_json_dict.keys())):
        task = key.split("/")[-1]
        if "scores" in original_json_dict[key].keys():
            # pretrained mpnet
            score_mpnet = original_json_dict[key]["scores"]["test"][0][
                "main_score"
            ]
            clean_dict[task] = score_mpnet

        else:
            continue

    return clean_dict

In [ ]:
import_path = (
    Path("embeddings_mpnet")  # + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
    / Path("results_mpnet_crops_finetuning")
)
mpnet_Results = create_mteb_dict(
    load_mteb_results(variables_path / import_path)
)
mpnet_Results

{'ArguAna': 0.50645,
 'ArxivClusteringP2P': 0.3831448182125486,
 'AskUbuntuDupQuestions': 0.5309807492868124,
 'BiorxivClusteringP2P': 0.3239812131713279,
 'MedrxivClusteringP2P': 0.30846324327144165,
 'MindSmallReranking': 0.3016545622166566,
 'NFCorpus': 0.18062,
 'NQ': 0.20662,
 'RedditClusteringP2P': 0.5585157919463574,
 'SCIDOCS': 0.13043,
 'STS12': 0.4910681280246965,
 'STS13': 0.7369556305640635,
 'STS14': 0.6407907818531218,
 'STS15': 0.724794851067155,
 'STS16': 0.7595258391529889,
 'STSBenchmark': 0.7170197143136364,
 'SciDocsRR': 0.7361146962017713,
 'SciFact': 0.47256,
 'StackExchangeClusteringP2P': 0.31285921622475893,
 'TRECCOVID': 0.30121,
 'TwentyNewsgroupsClustering': 0.27660299969511837}

In [ ]:
import_path = (
    Path("embeddings_sbert")  # + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
    / Path("results_sbert")
)
mpnet_Results = create_mteb_dict(
    load_mteb_results(variables_path / import_path)
)
mpnet_Results

{'ArguAna': 0.46521,
 'ArxivClusteringP2P': 0.48122865602904,
 'BiorxivClusteringP2P': 0.3931764814708352,
 'MedrxivClusteringP2P': 0.35588148423336013,
 'MindSmallReranking': 0.309694534640939,
 'RedditClusteringP2P': 0.5659670326945667,
 'SCIDOCS': 0.23765,
 'STS12': 0.7263429726307248,
 'STS13': 0.8348499016384896,
 'STS14': 0.7800011909623369,
 'STS15': 0.8566280012440247,
 'STS16': 0.8003003704871307,
 'STSBenchmark': 0.8342190421330611,
 'SciDocsRR': 0.8865396986895777,
 'StackExchangeClusteringP2P': 0.3431309861181532,
 'TwentyNewsgroupsClustering': 0.5006551061828595}

#### All tasks from 4 modalities

In [ ]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [ ]:
# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet = SentenceTransformer(modules=new_modules)

In [ ]:
tasks = mteb.get_tasks(
    task_types=["Clustering", "Retrieval", "Reranking", "STS"],
    languages=["eng"],
)
print(tasks)
evaluation = mteb.MTEB(tasks=tasks[28:])

MTEBTasks(WikiCitiesClustering(name='WikiCitiesClustering', languages=['eng']), MasakhaNEWSClusteringP2P(name='MasakhaNEWSClusteringP2P', languages=['eng']), MasakhaNEWSClusteringS2S(name='MasakhaNEWSClusteringS2S', languages=['eng']), ArXivHierarchicalClusteringP2P(name='ArXivHierarchicalClusteringP2P', languages=['eng']), ArXivHierarchicalClusteringS2S(name='ArXivHierarchicalClusteringS2S', languages=['eng']), BigPatentClusteringFast(name='BigPatentClustering.v2', languages=['eng']), BiorxivClusteringP2PFast(name='BiorxivClusteringP2P.v2', languages=['eng']), BiorxivClusteringS2SFast(name='BiorxivClusteringS2S.v2', languages=['eng']), MedrxivClusteringP2PFast(name='MedrxivClusteringP2P.v2', languages=['eng']), MedrxivClusteringS2SFast(name='MedrxivClusteringS2S.v2', languages=['eng']), RedditFastClusteringS2S(name='RedditClustering.v2', languages=['eng']), RedditFastClusteringP2P(name='RedditClusteringP2P.v2', languages=['eng']), StackExchangeClusteringFast(name='StackExchangeCluster

In [ ]:
for num, task in enumerate(tasks):
    print(num, task)
    # task..metadata.name
# len(tasks)

0 WikiCitiesClustering(name='WikiCitiesClustering', languages=['eng'])
1 MasakhaNEWSClusteringP2P(name='MasakhaNEWSClusteringP2P', languages=['eng'])
2 MasakhaNEWSClusteringS2S(name='MasakhaNEWSClusteringS2S', languages=['eng'])
3 ArXivHierarchicalClusteringP2P(name='ArXivHierarchicalClusteringP2P', languages=['eng'])
4 ArXivHierarchicalClusteringS2S(name='ArXivHierarchicalClusteringS2S', languages=['eng'])
5 BigPatentClusteringFast(name='BigPatentClustering.v2', languages=['eng'])
6 BiorxivClusteringP2PFast(name='BiorxivClusteringP2P.v2', languages=['eng'])
7 BiorxivClusteringS2SFast(name='BiorxivClusteringS2S.v2', languages=['eng'])
8 MedrxivClusteringP2PFast(name='MedrxivClusteringP2P.v2', languages=['eng'])
9 MedrxivClusteringS2SFast(name='MedrxivClusteringS2S.v2', languages=['eng'])
10 RedditFastClusteringS2S(name='RedditClustering.v2', languages=['eng'])
11 RedditFastClusteringP2P(name='RedditClusteringP2P.v2', languages=['eng'])
12 StackExchangeClusteringFast(name='StackExchange

In [ ]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_crops_finetuning",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Reranking

- AskUbuntuDupQuestions, s2s

- MindSmallReranking, s2s

- SciDocsRR, s2s

...

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [ ]:
for task in results:
    print(task, task.scores["test"][0]["main_score"])

In [ ]:
import os
import json


def load_mteb_results(directory):
    results = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    data = json.load(f)

                # Create a key based on the relative path
                relative_path = os.path.relpath(file_path, directory)
                key = os.path.splitext(relative_path)[
                    0
                ]  # Remove the .json extension
                results[key] = data

    return results

In [ ]:
def create_mteb_dict(original_json_dict):
    clean_dict = dict()
    for key in np.sort(list(original_json_dict.keys())):
        task = key.split("/")[-1]
        if "scores" in original_json_dict[key].keys():
            # pretrained mpnet
            score_mpnet = original_json_dict[key]["scores"]["test"][0][
                "main_score"
            ]
            clean_dict[task] = score_mpnet

        else:
            continue

    return clean_dict

In [ ]:
import_path = (
    Path("embeddings_mpnet")  # + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
    / Path("results_mpnet_crops_finetuning")
)
mpnet_Results = create_mteb_dict(
    load_mteb_results(variables_path / import_path)
)
mpnet_Results

{'ArguAna': 0.50645,
 'ArxivClusteringP2P': 0.3831448182125486,
 'AskUbuntuDupQuestions': 0.5309807492868124,
 'BiorxivClusteringP2P': 0.3239812131713279,
 'MedrxivClusteringP2P': 0.30846324327144165,
 'MindSmallReranking': 0.3016545622166566,
 'NFCorpus': 0.18062,
 'NQ': 0.20662,
 'RedditClusteringP2P': 0.5585157919463574,
 'SCIDOCS': 0.13043,
 'STS12': 0.4910681280246965,
 'STS13': 0.7369556305640635,
 'STS14': 0.6407907818531218,
 'STS15': 0.724794851067155,
 'STS16': 0.7595258391529889,
 'STSBenchmark': 0.7170197143136364,
 'SciDocsRR': 0.7361146962017713,
 'SciFact': 0.47256,
 'StackExchangeClusteringP2P': 0.31285921622475893,
 'TRECCOVID': 0.30121,
 'TwentyNewsgroupsClustering': 0.27660299969511837}

In [ ]:
import_path = (
    Path("embeddings_sbert")  # + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
    / Path("results_sbert")
)
mpnet_Results = create_mteb_dict(
    load_mteb_results(variables_path / import_path)
)
mpnet_Results

{'ArguAna': 0.46521,
 'ArxivClusteringP2P': 0.48122865602904,
 'BiorxivClusteringP2P': 0.3931764814708352,
 'MedrxivClusteringP2P': 0.35588148423336013,
 'MindSmallReranking': 0.309694534640939,
 'RedditClusteringP2P': 0.5659670326945667,
 'SCIDOCS': 0.23765,
 'STS12': 0.7263429726307248,
 'STS13': 0.8348499016384896,
 'STS14': 0.7800011909623369,
 'STS15': 0.8566280012440247,
 'STS16': 0.8003003704871307,
 'STSBenchmark': 0.8342190421330611,
 'SciDocsRR': 0.8865396986895777,
 'StackExchangeClusteringP2P': 0.3431309861181532,
 'TwentyNewsgroupsClustering': 0.5006551061828595}

### Fine-tuned MPNet (SimCSE-like)

In [ ]:
# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_simcse_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet_simcse = SentenceTransformer(modules=new_modules)

In [ ]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet_simcse,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_simcse_finetuning",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, s2p

- ArguAna, s2p

STS

...

Batches:   0%|          | 0/894 [00:00<?, ?it/s]

Clustering:  60%|█████████████████████████████████████▏                        | 6/10 [12:15<08:45, 131.48s/it]

Batches:   0%|          | 0/2161 [00:00<?, ?it/s]

Clustering:  70%|███████████████████████████████████████████▍                  | 7/10 [14:33<06:41, 133.80s/it]

Batches:   0%|          | 0/2109 [00:00<?, ?it/s]

### Pre-trained SBERT

In [ ]:
model_names = [
    "SBERT",
]


model_paths = [
    "sentence-transformers/all-mpnet-base-v2",
]

In [ ]:
pretrained_sbert = SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2"
)

In [ ]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_sbert,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

In [ ]:
results[0].scores  # scores

{'test': [{'v_measure': 0.48122865602904,
   'v_measure_std': 0.1420737036434218,
   'v_measures': [0.47747775788196556,
    0.46992600355904723,
    0.4827140346059144,
    0.4885767563767572,
    0.4894702123648364,
    0.48157088445711504,
    0.4622690570510022,
    0.47304447734321053,
    0.4808516368387714,
    0.479967635923842,
    0.5684267022079372,
    0.5710113196939407,
    0.5681439209719611,
    0.5706508654041879,
    0.5695516421163074,
    0.5742516683510883,
    0.5706572886532014,
    0.5693702625249434,
    0.564890036535079,
    0.5707430582833042,
    0.529674654063362,
    0.2553072011727571,
    0.4713293397649489,
    0.41335919030567564,
    0.3528146126776203,
    0.28343364210120076,
    0.3028504710337586,
    0.22292767529384594,
    0.318104027431572,
    1.0,
    0.284722301911086],
   'main_score': 0.48122865602904,
   'hf_subset': 'default',
   'languages': ['eng-Latn']}]}

#### More tasks

In [ ]:
new_tasks = [
    "AskUbuntuDupQuestions",  # reranking
    "NFCorpus",  # retrieval, ordered by relevance
    "NQ",
    "TRECCOVID",
    "SciFact",
]

In [ ]:
tasks = mteb.get_tasks(tasks=new_tasks)
evaluation = mteb.MTEB(tasks=tasks)

In [ ]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_sbert,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Reranking

- AskUbuntuDupQuestions, s2s

Retrieval

- NFCorpus, s2p

...

Batches:   0%|          | 0/167 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
for task in results:
    print(task.task_name, task.scores["test"][0]["main_score"])

AskUbuntuDupQuestions 0.6585199009344481
NFCorpus 0.33289
NQ 0.5045
TRECCOVID 0.51305
SciFact 0.6557


mpnet
AskUbuntuDupQuestions 0.5309807492868124
NFCorpus 0.18062
NQ 0.20662
TRECCOVID 0.30121
SciFact 0.47256

### Pre-trained SimCSE

In [ ]:
model_names = [
    "SimCSE",
]


model_paths = [
    "princeton-nlp/unsup-simcse-bert-base-uncased",
]

In [ ]:
pretrained_simcse = SentenceTransformer(model_paths[0])

/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_simcse,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Reranking

- SciDocsRR, s2s

- MindSmallReranking, s2s

Retrieval

...

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

In [ ]:
results[0].scores  # scores

{'test': [{'v_measure': 0.28807582656487557,
   'v_measure_std': 0.016650520720206934,
   'v_measures': [0.2780986581221716,
    0.2696831576885149,
    0.2692357874378456,
    0.26793095576309434,
    0.27492677329510967,
    0.3124345646729936,
    0.3016249511045954,
    0.2992775441140408,
    0.30546110723744707,
    0.302084766212942],
   'main_score': 0.28807582656487557,
   'hf_subset': 'default',
   'languages': ['eng-Latn']}]}

### Random embedding layer

#### Fine-tune model and save

In [ ]:
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model_random = BertModel(configuration)

# Accessing the model configuration
configuration = model_random.config

In [ ]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Extract the embedding layer
embedding_layer = model_random.embeddings.word_embeddings
# new model
embedding_only_model_random = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda
CPU times: user 38.6 ms, sys: 11.8 ms, total: 50.4 ms
Wall time: 161 ms


In [ ]:
# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract,
    tokenizer,
    device,
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
(
    embedding_only_model_random_trained,
    losses,
) = train_loop_without_eval_embedding_layer(
    embedding_only_model_random,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=10,
    lr=5e-1,
)

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

In [ ]:
# Save the model and tokenizer
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_layer"
)
embedding_only_model_random_trained.save_pretrained(
    variables_path / output_dir
)

#### Evaluate 
All above does not need to be run, one only needs to import the trained model checkpoint

In [ ]:
# fix random seeds
fix_all_seeds()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# load model
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_layer"
)
embedding_only_model_random_trained = EmbeddingOnlyModel.from_pretrained(
    variables_path / output_dir
)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
# embedding sentence model
embedding_only_model_random_trained.to("cpu")
embedding_only_sentence_model_random = MyEmbeddingSentenceModel(
    embedding_only_model_random_trained, tokenizer
)

In [ ]:
tasks = mteb.get_tasks(
    tasks=[
        "ArxivClusteringP2P",
        "BiorxivClusteringP2P",
        "MedrxivClusteringP2P",
        "RedditClusteringP2P",
        # "TwentyNewsgroupsClustering",
        "StackExchangeClusteringP2P",
        "SciDocsRR",
        "MindSmallReranking",
        "SCIDOCS",
        "ArguAna",
        "STS12",
        "STS13",
        "STS14",
        "STS15",
        "STS16",
        "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

In [ ]:
saving_path = (
    Path("embeddings_random_bert")
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    embedding_only_sentence_model_random,
    output_folder=variables_path
    / saving_path
    / f"results_embedding_layer_random_bert",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, s2p

- ArguAna, s2p

STS

...

- RedditClusteringP2P, p2p

Reranking

- SciDocsRR, s2s

- MindSmallReranking, s2s

Clustering:   0%|                                                                       | 0/31 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possibl

## Random BERT 
new 28-01-2025

### Fine-tune and save it

#### 1 epoch

In [ ]:
%%time
model_name = "random_bert"

# fix random seeds
fix_all_seeds()

# set up model
print("Model: ", model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

## randomly initialized model
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()
# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertModel(configuration)
# Accessing the model configuration
configuration = model.config


# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract, tokenizer, device, n_cons_sntcs=2, seed=42
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
random_bert_trained_crops, losses = train_loop_without_eval(
    model,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
)

# save
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_crops_finetuning"
)
random_bert_trained_crops.save_pretrained(variables_path / output_dir)


print("----------------------------")

Model:  random_bert
Running on device: cuda


  0%|          | 0/368 [00:00<?, ?it/s]

----------------------------
CPU times: user 5min 38s, sys: 59.8 s, total: 6min 37s
Wall time: 5min 17s


#### 10 epochs

In [ ]:
%%time
# STILL TO RUN
model_name = "random_bert"

# fix random seeds
fix_all_seeds()

# set up model
print("Model: ", model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

## randomly initialized model
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()
# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertModel(configuration)
# Accessing the model configuration
configuration = model.config


# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract, tokenizer, device, n_cons_sntcs=2, seed=42
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
random_bert_trained_crops, losses = train_loop_without_eval(
    model,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=10,
    lr=2e-5,
)

# save
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_crops_finetuning_10_epochs"
)
random_bert_trained_crops.save_pretrained(variables_path / output_dir)


print("----------------------------")

Model:  random_bert
Running on device: cuda


  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

...

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

----------------------------
CPU times: user 44min 39s, sys: 6min 43s, total: 51min 22s
Wall time: 49min 55s


### Evaluation

In [ ]:
from sentence_transformers import SentenceTransformer, models


def sentence_transformer_wrapper(custom_base_model, custom_tokenizer):
    """
    Wraps any transformer into a SentenceTransformer (only object differences, does not mean whether the model is trained to represent sentence sor not).
    Needed because MTEB only takes models with the structure of SentenceTransformers for evaluation.
    """

    # Create a new SentenceTransformer model
    new_modules = []

    # Wrap your custom base model in a Transformer module
    transformer_model = models.Transformer(
        model_name_or_path="bert-base-uncased",  # None gives an error,
        # so I initialize another model that
        # will be substituted by my custom model below
        max_seq_length=384,  # You can adjust this as needed
        do_lower_case=False,  # Adjust based on your tokenizer
    )
    # Replace the auto_model in the Transformer wrapper with your custom model
    transformer_model.auto_model = custom_base_model
    # Set the tokenizer
    transformer_model.tokenizer = custom_tokenizer

    # Add the wrapped model as the first module
    new_modules.append(transformer_model)

    # Add Pooling layer
    pooling_model = models.Pooling(
        word_embedding_dimension=custom_base_model.config.hidden_size,
        pooling_mode_cls_token=False,
        pooling_mode_mean_tokens=True,
        pooling_mode_max_tokens=False,
        pooling_mode_mean_sqrt_len_tokens=False,
    )
    new_modules.append(pooling_model)

    # Add Normalize layer
    new_modules.append(models.Normalize())

    # Create the new SentenceTransformer model
    my_sentence_transformer = SentenceTransformer(modules=new_modules)

    return my_sentence_transformer

In [ ]:
# Load your custom base model and tokenizer
checkpoint_dir = Path(
    # "updated_dataset/model_checkpoints/random_bert_crops_finetuning"
    "updated_dataset/model_checkpoints/random_bert_crops_finetuning_10_epochs"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

finetuned_random_bert = sentence_transformer_wrapper(
    custom_base_model, custom_tokenizer
)

In [ ]:
type(finetuned_random_bert)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [ ]:
tasks = mteb.get_tasks(
    tasks=[
        "ArxivClusteringP2P",
        "BiorxivClusteringP2P",
        "MedrxivClusteringP2P",
        "RedditClusteringP2P",
        # "TwentyNewsgroupsClustering",
        "StackExchangeClusteringP2P",
        "SciDocsRR",
        "MindSmallReranking",
        "SCIDOCS",
        "ArguAna",
        "STS12",
        "STS13",
        "STS14",
        "STS15",
        "STS16",
        "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

In [ ]:
model_name = "random_bert"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_random_bert,
    output_folder=variables_path / saving_path
    # / f"results_{model_name.lower()}_crops_finetuning_1_epoch",
    / f"results_{model_name.lower()}_crops_finetuning_10_epochs",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Clustering

- ArxivClusteringP2P, p2p

- BiorxivClusteringP2P, p2p

- MedrxivClusteringP2P, p2p

...

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]